In [1]:
import pickle
with open('train.pkl', 'rb') as f:
    x, y = pickle.load(f)

print(x.shape, y.shape)  # (number_of_samples, height, width) (number_of_samples,)

(30000, 1, 150, 150) (30000,)


In [2]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=3)

In [3]:
import numpy as np

# Reshape X_train to the correct shape and duplicate the grayscale channel across 3 channels
X_train_rgb = np.repeat(x, 3, axis=1)

# Ensure the shape is now (24000, 3, 150, 150)
print(X_train_rgb.shape)

# If using TensorFlow, which expects the channel last format, you might need to transpose the axes
X_train_rgb = np.transpose(X_train_rgb, (0, 2, 3, 1))

# Ensure the shape is now (24000, 150, 150, 3) for TensorFlow
print(X_train_rgb.shape)


(30000, 3, 150, 150)
(30000, 150, 150, 3)


In [4]:
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import train_test_split


# Example loading data
# X = np.load('your_dataset.npy')
# y = np.load('your_labels.npy')

# Resize your images to 224x224 if they are not already (this might require additional code)
# X_resized = resize_images(X)  # This is a placeholder function

# Preprocess input according to ResNet's requirements
X_processed = preprocess_input(X_train_rgb)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [6]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the VGG16 model, pre-trained on ImageNet, without the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)  # Add a global spatial average pooling layer
x = Dense(1024, activation='relu')(x)  # Add a fully-connected layer with 1024 units and ReLU activation
predictions = Dense(3, activation='softmax')(x)  # Create the output layer with 3 units and a softmax activation

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)




58889256/58889256 [==============================] - 6s 0us/step


In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [9]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


Epoch 1/10


750/750 [==============================] - 630s 839ms/step - loss: 1.1319 - accuracy: 0.3329 - val_loss: 1.1149 - val_accuracy: 0.3327
Epoch 2/10
750/750 [==============================] - 602s 802ms/step - loss: 1.1153 - accuracy: 0.3343 - val_loss: 1.1089 - val_accuracy: 0.3337
Epoch 3/10
750/750 [==============================] - 601s 801ms/step - loss: 1.1052 - accuracy: 0.3349 - val_loss: 1.1112 - val_accuracy: 0.3337
Epoch 4/10
750/750 [==============================] - 599s 799ms/step - loss: 1.1022 - accuracy: 0.3300 - val_loss: 1.1024 - val_accuracy: 0.3327
Epoch 5/10
750/750 [==============================] - ETA: 0s - loss: 1.0996 - accuracy: 0.3318

KeyboardInterrupt: 